In [ ]:
"""
Refine topic modeling

- Model: NMF (Tf-idf)
- Data: divide all transcripts into subgroups
- Goal: see the evolution of topic trends

"""

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from collections import Counter

from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

from wordcloud import WordCloud

from gensim import matutils, models
import scipy.sparse

import re
import string

import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer



%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/katiehuang/Desktop/metis/projects/onl_ds5_project_4/py')
from word_cloud import *
from clean import *
import importlib

In [2]:
# Let's read in our transcripts and document-term matrix
speech_df_raw = pd.read_pickle('../dump/speech_df')
speech_df = pd.read_pickle('../dump/speech_clean_lemma')
data = pd.read_pickle('../dump/data_dtm_lemma.pkl')
tdm = data.transpose()
tdm.shape

(36156, 441)

## 1. Divide the transcripts base on time
Aim to divied into 5 parts; each ~ 90 speeches.
1. Pre 2003
2. 2003-2009
3. 2009-2012
4. 2012-2015
5. 2015-

In [3]:
# Examine years
sorted_year = speech_df.sort_values('year')
sorted_year

,speaker,year,transcript,length
440,GARRISON KEILLOR,0,its an honor to be with so many smart people a...,8063
439,RALPH WALDO,1838,in this refulgent summer it have be a luxury t...,40403
438,OPRAH WINFREY,1918,thank you wallis annenberg and a special thank...,15301
437,FRANKLIN D ROOSEVELT,1932,for me as for you this be a day of honorable a...,16017
436,CARRIE CHAPMAN,1936,i bring a message to sweet briar college and e...,22942
...,...,...,...,...
4,JESMYN WARD,2018,persist be patient be well good morning it be ...,14063
3,MINDY KALING,2018,good morning to the class of the faculty the p...,15391
2,JON B. FISHER,2018,thank you very much my father commencement spe...,8544
1,ABBY WAMBACK,2018,failure be not something to be ashamed of its ...,15866


In [4]:
# Update correct year
sorted_year.loc[440,'year'] = 2015
sorted_year = sorted_year.sort_values('year')

In [5]:
# Count number of speech in each time slot
speech_df_1 = sorted_year[sorted_year.year<2003]
speech_df_2 = sorted_year[(sorted_year.year>= 2003)&(sorted_year.year<2009)]
speech_df_3 = sorted_year[(sorted_year.year>= 2009)&(sorted_year.year<2012)]
speech_df_4 = sorted_year[(sorted_year.year>= 2012)&(sorted_year.year<2015)]
speech_df_5 = sorted_year[sorted_year.year>= 2015]

speech_df_1.shape[0],\
speech_df_2.shape[0],\
speech_df_3.shape[0],\
speech_df_4.shape[0],\
speech_df_5.shape[0],

(87, 90, 94, 101, 69)

## 2. Preparation for topic modeling

In [85]:
def topic_modeling(speech_df_x,column_name,num_topic):
    """
    For a given df and column_name (e.g.:'transcript' for all text, 'nouns' for nouns only),
    return the topic_word dataframe
    
    """
    
    # Prepare Tf-idf
    tfidf = TfidfVectorizer()
    csr_mat = tfidf.fit_transform(speech_df_x[column_name])
    doc_word = tfidf_dtm(speech_df_x,column_name)

    # Modeling
    nmf_model = NMF(num_topic, max_iter=1000)
    doc_topic = nmf_model.fit_transform(doc_word)

    # Use components in NMF model to find the top 10 words for a given topic
    topics = nmf_model.components_.argsort(axis=1)[:,-1:-11:-1]

    # Create topic_worrd df
    words = doc_word.columns
    topic_words = [[words[index] for index in topic] for topic in topics]
    topic_word_df = pd.DataFrame(topic_words,index=['Topic #' + '{:02d}'.format(i+1) for i in range(num_topic)])
    
    return topic_word_df

## 3. Topic modeling for each time frame

In [111]:
num_topic = 4

### A. Pre 2003

In [112]:
first = speech_df_raw[speech_df_raw.year<2003].copy()

In [113]:
# Wash the transcripts!
speech_df_x = wash(first,'transcript').copy()

#### All text

In [114]:
# topic_modeling(speech_df_x,'transcript',6)

#### Nouns only

In [115]:
topic_df = topic_modeling(speech_df_x,'nouns',num_topic)
topic_df

In [116]:
# Pickle
topic_df.to_pickle('../dump/to_nx_topic_df_1')

### B. 2003-2009

In [117]:
second = speech_df_raw[(speech_df_raw.year>= 2003)&(speech_df_raw.year<2009)].copy()

In [118]:
# Wash the transcripts!
speech_df_x = wash(second,'transcript').copy()

#### All text

In [119]:
# topic_modeling(speech_df_x,'transcript',6)

#### Nouns only

In [120]:
topic_df = topic_modeling(speech_df_x,'nouns',num_topic)
topic_df

,0,1,2,3,4,5,6,7,8,9
Topic #01,education,degree,community,dream,family,mother,opportunity,sister,thank,doctor
Topic #02,game,doors,kid,ball,door,husband,neighborhood,coach,country,dream
Topic #03,fear,idea,parent,age,failure,mind,sense,word,experience,generation
Topic #04,company,selfconfidence,business,percent,confidence,rule,purpose,career,challenge,engineer


In [121]:
# Pickle
topic_df.to_pickle('../dump/to_nx_topic_df_2')

### C. 2009-2012

In [122]:
third = speech_df_raw[(speech_df_raw.year>= 2009)&(speech_df_raw.year<2012)].copy()

In [123]:
# Wash the transcripts!
speech_df_x = wash(third,'transcript').copy()

#### All text

In [124]:
# topic_modeling(speech_df_x,'transcript',6)

#### Nouns only

In [125]:
topic_df = topic_modeling(speech_df_x,'nouns',num_topic)
topic_df

,0,1,2,3,4,5,6,7,8,9
Topic #01,education,money,parent,dream,heart,business,future,value,place,company
Topic #02,women,genius,conversation,success,workforce,balance,voice,belong,worlds,girl
Topic #03,bird,dream,obstacles,drift,polluters,thuddy,compliance,grammar,instructors,bust
Topic #04,policy,grid,dream,commencement,mule,blue,diploma,man,address,months


In [126]:
# Pickle
topic_df.to_pickle('../dump/to_nx_topic_df_3')

### D. 2012-2015

In [127]:
fourth = speech_df_raw[(speech_df_raw.year>= 2012)&(speech_df_raw.year<2015)].copy()

In [128]:
# Wash the transcripts!
speech_df_x = wash(fourth,'transcript').copy()

#### All text

In [129]:
# topic_modeling(speech_df_x,'transcript',6)

#### Nouns only

In [130]:
topic_df = topic_modeling(speech_df_x,'nouns',num_topic)
topic_df

,0,1,2,3,4,5,6,7,8,9
Topic #01,dream,parent,fear,moment,story,year,friends,kid,family,room
Topic #02,advice,hater,voice,ignorance,women,silence,hiphop,book,marriage,pigeon
Topic #03,generation,women,success,country,place,challenge,education,history,family,children
Topic #04,substitute,phone,communication,someones,distract,dimension,entangle,responses,compassion,conversation


In [131]:
# Pickle
topic_df.to_pickle('../dump/to_nx_topic_df_4')

### E. 2015-

In [132]:
fifth = speech_df_raw[(speech_df_raw.year>= 2015)].copy()

In [133]:
# Wash the transcripts!
speech_df_x = wash(fifth,'transcript').copy()

#### All text

In [134]:
# topic_modeling(speech_df_x,'transcript',6)

#### Nouns only

In [135]:
topic_df = topic_modeling(speech_df_x,'nouns',num_topic)
topic_df

,0,1,2,3,4,5,6,7,8,9
Topic #01,advice,film,speech,voice,movie,play,career,money,letter,year
Topic #02,purpose,sense,generation,communities,freedom,company,project,society,community,millions
Topic #03,mother,shape,gender,women,prize,sign,feminism,makeup,privilege,earth
Topic #04,question,resilience,company,days,children,joy,step,family,cancer,success


In [136]:
# Pickle
topic_df.to_pickle('../dump/to_nx_topic_df_5')